# Convert To Filtered Spectrogram

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import os
from scipy.signal import butter, lfilter

Parameters

In [5]:
data_directory = '../data/pilot01_data/'
target_directory = '1'
results_directory = 'results'

folder_path = os.path.join(data_directory, target_directory)
results_path = os.path.join(data_directory, results_directory, target_directory)
os.makedirs(results_path, exist_ok=True)
filter_start = 900
filter_stop = 9500


In [ ]:
def iterate_through_wav(folder_path, results_folder_path, filter_start, filter_stop, recursive=True, plot=False):
    """
    Iterate through wav files within target folder, get filtered spectrogram and save data
    """

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            print(file_path)
            filtered_spectrogram, _, _ = get_filtered_spectrogram(file_path, filter_start, filter_stop, plot)

            target_file = f"{os.path.splitext(file)[0]}.npy"
            results_file_path = os.path.join(results_folder_path, target_file)
            save_data(results_file_path, filtered_spectrogram)

        if not recursive:
            # Exit the loop after the top-level folder if not recursive
            break
    
def save_data(file_path, data):
    """ 
    Saves numpy array as csv
    """
    np.savetxt(file_path, data)

def get_filtered_spectrogram(file_path, filter_start, filter_stop, plot=False):
    """
    Obtains data from file_path, converts to spectrogram and then filters for desired range
    """

    # Read the .wav file
    sample_rate, data = wavfile.read(file_path)
    # Handle stereo audio (take one channel)
    if len(data.shape) == 2:
        data = data[:, 0]

    spectrogram, freqs, times, _ = plt.specgram(data, Fs=sample_rate, )

    if plot:
        plot_spectrogram(spectrogram, freqs, times)

    filtered_spectrogram, freqs = filter_spectrogram(spectrogram, freqs, filter_start, filter_stop)

    if plot:
        plot_spectrogram(filtered_spectrogram, freqs, times)

    return filtered_spectrogram, freqs, times

def filter_spectrogram(spectrogram, freqs, filter_start, filter_stop):
    freq_index = np.where((freqs >= filter_start) & (freqs <= filter_stop))[0]

    filtered_spectrogram = spectrogram[freq_index, :]
    filtered_freqs = freqs[freq_index]

    return filtered_spectrogram, filtered_freqs


def plot_spectrogram(spectrogram, freqs, times):
    plt.imshow(10 * np.log10(spectrogram), aspect='auto', extent=[times.min(), times.max(), freqs.min(), freqs.max()], origin='lower')
    plt.colorbar(label='Intensity [dB]')
    plt.xlabel('Time [s]')
    plt.ylabel('Frequency [Hz]')
    plt.title(f'Spectrogram of {"blah"}')
    plt.show()

In [ ]:

iterate_through_wav(folder_path, results_path, filter_start, filter_stop, plot=False)